In [ ]:
qlogin -q short.qg -l gpu=1[affinity=true],gputype=rtx8000
# qlogin -q short.qg -l gpu=1[affinity=true],gputype=a100

In [ ]:
#!/bin/bash

cd /well/win/users/hsv459/multi-age-mapper

module purge
module load Python/3.7.4-GCCcore-8.3.0
module load CUDA/10.1.243-GCC-8.3.0
module load cudnn/7.6.5.32-10.1
# module add fsl/5.0.11

# VISIBLE_CUDA_DEVICES=0,1

# determine Ivybridge or Skylake compatibility on this node
CPU_ARCHITECTURE=$(/apps/misc/utils/bin/get-cpu-software-architecture.py)
# CPU_ARCHITECTURE=general

# Error handling
if [[ ! $? == 0 ]]; then
  echo "Fatal error: Please send the following information to the BMRC team: Could not determine CPU software architecture on $(hostname)"
  exit 1
fi

# Activate the ivybridge or skylake version of your python virtual environment
source /well/win/users/hsv459/python/functionmapper-${CPU_ARCHITECTURE}/bin/activate
# source /well/win/users/hsv459/python/functionmapper-skylake/bin/activate

# continue to use your python venv as normal
   
# ipython



In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from solver import Solver
from utils.data_utils import get_datasets_dynamically, get_test_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations


# Set the default floating point tensor type to FloatTensor
torch.set_default_tensor_type(torch.FloatTensor)

def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data
    else:
        print('ERROR: Invalid Flag')
        return None
    
settings_file_name = 'MMx-1.ini'

settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

train_data, validation_data = load_data_dynamically(data_parameters=data_parameters, flag='train')

train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=1
)

i=0
for inputs, targets in train_loader:
    if i>1:
        break
    print(type(inputs), type(targets))
    print('----------')
    i+=1
        
print(inputs[0][:,80,90,50:55])
print(inputs[1][:,80,90,50:55])
print(inputs[2][:,80,90,50:55])
# print(np.divide(inputs[0][:,80,90,50:55], inputs[1][:,80,90,50:55]))

# tensor([[0.6915, 0.5228, 0.5321, 0.5967, 0.6413]])
# tensor([[0.4916, 0.6317, 0.6725, 0.6307, 0.5606]])
# tensor([[1.4067, 0.8276, 0.7912, 0.9461, 1.1440]])
# tensor([[1.4067, 0.8276, 0.7912, 0.9461, 1.1440]])